In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Set up
import sys

sys.path.append("/home/dchen/BA_CH_EN/")

# Basics
import tensorflow as tf
import xarray as xr

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.DRN.DRN_load_preds import *
from src.models.EMOS_local.EMOS_local_load_preds import *  # Load EMOS_global_preds
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_preds import *  # Load EMOS_global_preds
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

2023-06-29 15:51:35.010034: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-29 15:51:35.010055: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Set Up

In [3]:
lead_time = 0
var_num = 2
var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]

<IPython.core.display.Javascript object>

### 1. Load Data

In [4]:
land_sea_mask_dummy = np.load(
    "/Data/Delong_BA_Data/land_sea_mask_dummy/land_sea_mask_dummy.npy"
)

<IPython.core.display.Javascript object>

In [5]:
# Load all training data of each variable
train_var_denormed = ldpd.load_data_all_train_proc_denorm()

# Split the loaded data into features (X) and target (y)
# also adjusts for lead_time
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    train_var_denormed
)

# Preprocess the features for Neural Network and scale them
drn_X_train_lead_array, drn_embedding_train_lead_array = make_X_array(
    dat_X_train_lead_all_denorm, lead_time
)

# Reshape target values into a 1D array
t2m_y_train = dat_y_train_lead_all_denorm[var_num][lead_time].values.flatten()

<IPython.core.display.Javascript object>

### 2. Make new Dataset

In [59]:
land_sea_mask_input = np.tile(land_sea_mask_dummy.flatten(), 1429)

<IPython.core.display.Javascript object>

In [66]:
drn_X_train_lead_array_mask = np.column_stack(
    (drn_X_train_lead_array, land_sea_mask_input)
)

<IPython.core.display.Javascript object>

### 3. Make Model

In [78]:
drn_lead_model = build_emb_model(
    13,
    2,
    [256],
    5,
    15599,
    compile=True,
    optimizer="Adam",
    lr=0.001,
    loss=crps_cost_function,
    activation="relu",
)

<IPython.core.display.Javascript object>

In [79]:
drn_lead_model.fit(
    [drn_X_train_lead_array_mask, drn_embedding_train_lead_array],
    t2m_y_train,
    epochs=30,
    batch_size=1024,
    validation_split=0.25,
    verbose=1,
)

Epoch 1/30
16328/16328 [==============================] - 61s 4ms/step - loss: 5.3562 - val_loss: 0.4446
Epoch 2/30
16328/16328 [==============================] - 63s 4ms/step - loss: 0.4032 - val_loss: 0.3978
Epoch 3/30
16328/16328 [==============================] - 55s 3ms/step - loss: 0.3920 - val_loss: 0.3876
Epoch 4/30
  945/16328 [>.............................] - ETA: 49s - loss: 0.3874

KeyboardInterrupt: 

<IPython.core.display.Javascript object>